In [1]:
import numpy as np
import torch
from transformers import DPTFeatureExtractor, DPTForDepthEstimation
from PIL import Image
import os

# Depth Estimation

In [2]:
feature_extractor = DPTFeatureExtractor.from_pretrained("Intel/dpt-large")
model = DPTForDepthEstimation.from_pretrained("Intel/dpt-large")


def estimate_depth_dpt(image):
        # prepare image for the model
        encoding = feature_extractor(image, return_tensors="pt")
        # forward pass
        with torch.no_grad():
            outputs = model(**encoding)
            predicted_depth = outputs.predicted_depth
        # interpolate to original size
        prediction = torch.nn.functional.interpolate(
                            predicted_depth.unsqueeze(1),
                            size=image.size[::-1],
                            mode="bicubic",
                            align_corners=False,
                        ).squeeze()
        output = prediction.cpu().numpy()
        formatted = (output * 255 / np.max(output)).astype('uint8')
        img = Image.fromarray(formatted)
        #convert to cv2
        img = np.array(img)
        return img

/home/deep-sewer/.conda/envs/huggingface/lib/python3.10/site-packages/transformers/models/dpt/feature_extraction_dpt.py:28: FutureWarning: The class DPTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DPTImageProcessor instead.
  warnings.warn(
Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Loftr

In [3]:
from LoFTR.src.loftr import LoFTR, default_cfg

matcher = LoFTR(config=default_cfg)
matcher.load_state_dict(torch.load('LoFTR/weights/outdoor_ds.ckpt')['state_dict'])
def compute_matches_loftr(fname_depthA,fname_depthB,match_th=0.1):
 
    loftr_matcher = matcher.eval().cuda()
    
    img0 = torch.from_numpy(fname_depthA)[None][None].cuda() / 255.
    img1 = torch.from_numpy(fname_depthB)[None][None].cuda() / 255.
    batch = {'image0': img0, 'image1': img1}

    # Inference with LoFTR and get prediction
    with torch.no_grad():
        loftr_matcher(batch)
        mkpts0 = batch['mkpts0_f'].cpu().numpy()
        mkpts1 = batch['mkpts1_f'].cpu().numpy()
        mconf = batch['mconf'].cpu().numpy()
    
    mkpts0 = mkpts0[mconf>match_th]
    mkpts1 = mkpts1[mconf>match_th]
    mconf = mconf[mconf>match_th]
    return mkpts0, mkpts1, mconf


# Real vs BIM Matching

In [4]:

def real_bim_matching(real_img_file,bim_img_file):
    real_img = Image.open(real_img_file).convert("RGB").resize((640,480))
    bim_img = Image.open(bim_img_file).convert("RGB").resize((640,480))
    real_img_depth = estimate_depth_dpt(real_img)
    bim_img_depth = estimate_depth_dpt(bim_img)
    mkpts0, mkpts1, mconf = compute_matches_loftr(real_img_depth,bim_img_depth)
    return np.mean(mconf), len(mconf)



# Compute Image matching metrics

In [5]:
import pandas as pd
from tqdm import tqdm
folder_path_real_imgs ='data/Sam/wetransfer_02_images_xmpfiles_2023-11-07_1050/02_Images_XMPfiles'
folder_path_bim_imgs = 'data/Sam/wetransfer_02_images_xmpfiles_2023-11-07_1050/04_renderedImages'

df = pd.read_csv('ImageNames_List.csv')
df_results = pd.DataFrame(columns=['real_img','bim_img','n_matches','avg_score','mutual_information'])

print('Computing Matching Scores')
for i in tqdm(range(len(df))):
    real_img_file = os.path.join(folder_path_real_imgs,df.iloc[i][0]+'.jpeg')
    bim_img_file = os.path.join(folder_path_bim_imgs,df.iloc[i][1]+'.png')
    real_img_name = real_img_file.split('/')[-1]
    bim_img_name = bim_img_file.split('/')[-1]
    mean_matching_score, n_matches = real_bim_matching(real_img_file,bim_img_file)
    df_results = df_results.append({'real_img':real_img_name, 'bim_img':bim_img_name,'n_matches':n_matches,'avg_score':mean_matching_score},ignore_index=True)
# save the results into a csv file
df_results.to_csv('MatchingResults.csv',index=False)
    


Computing Matching Scores


  0%|          | 0/9 [00:00<?, ?it/s]/home/deep-sewer/Workspace/RealvsBim/LoFTR/src/loftr/utils/coarse_matching.py:246: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  [i_ids % data['hw0_c'][1], i_ids // data['hw0_c'][1]],
/home/deep-sewer/Workspace/RealvsBim/LoFTR/src/loftr/utils/coarse_matching.py:249: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mod